In [ ]:
import re
from io import BytesIO

import numpy as np
import pandas as pd
import requests
from PIL import Image

from utils import *

In [933]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [934]:
df = pd.read_csv('the_gioi_di_dong_v2.csv')

In [935]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 44 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   filename                    450 non-null    object
 1   filename_origin             450 non-null    object
 2   time_scraping               450 non-null    object
 3   Link                        450 non-null    object
 4   Tiêu đề                     450 non-null    object
 5   Đã bán                      422 non-null    object
 6   Giá hiện tại                414 non-null    object
 7   Giá cũ                      325 non-null    object
 8   Giảm giá (%)                245 non-null    object
 9   Công nghệ CPU:              450 non-null    object
 10  Số nhân:                    450 non-null    int64 
 11  Số luồng:                   450 non-null    object
 12  Tốc độ CPU:                 450 non-null    object
 13  Tốc độ tối đa:              450 non-null    object

In [936]:
detect_missing(df)

,Missing Values,Missing %
Màn hình cảm ứng:,418,92.888889
Tản nhiệt:,412,91.555556
Hiệu năng xử lý AI (TOPS):,347,77.111111
NPU:,339,75.333333
Khe đọc thẻ nhớ:,291,64.666667
Giảm giá (%),205,45.555556
Độ phủ màu:,168,37.333333
Giá cũ,125,27.777778
Tính năng khác:,73,16.222222
Giá hiện tại,36,8.000000


In [ ]:
target = "Hãng không công bố"

for col in df.columns:
    if df[col].astype(str).str.strip().str.lower().eq(target.lower()).any():
        print(f"{col}")
        df[col] = df[col].astype(str).replace(
            to_replace=r"(?i)^\s*hãng không công bố\s*$",
            value=np.nan,
            regex=True
        )

# Thay thế các chuỗi biểu thị giá trị thiếu bằng np.nan
df.replace(
    ["nan", "NaN", "null", "None", ""],
    np.nan,
    inplace=True
)


Số luồng:
Tốc độ CPU:
Tốc độ tối đa:
Loại RAM:
Tốc độ Bus RAM:
Hỗ trợ RAM tối đa:
Tần số quét:
Hiệu năng xử lý AI (TOPS):


In [938]:
df = df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip().lower() == 'hãng không công bố' else x)

C:\Users\ACER\AppData\Local\Temp\ipykernel_36648\2068020170.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip().lower() == 'hãng không công bố' else x)


In [939]:
detect_missing(df)

,Missing Values,Missing %
Màn hình cảm ứng:,418,92.888889
Tản nhiệt:,412,91.555556
Hiệu năng xử lý AI (TOPS):,391,86.888889
NPU:,339,75.333333
Khe đọc thẻ nhớ:,291,64.666667
Giảm giá (%),205,45.555556
Độ phủ màu:,168,37.333333
Giá cũ,125,27.777778
Số luồng:,116,25.777778
Tốc độ Bus RAM:,113,25.111111


In [940]:
df.columns

Index(['filename', 'filename_origin', 'time_scraping', 'Link', 'Tiêu đề',
       'Đã bán', 'Giá hiện tại', 'Giá cũ', 'Giảm giá (%)', 'Công nghệ CPU:',
       'Số nhân:', 'Số luồng:', 'Tốc độ CPU:', 'Tốc độ tối đa:', 'RAM:',
       'Loại RAM:', 'Tốc độ Bus RAM:', 'Hỗ trợ RAM tối đa:', 'Ổ cứng:',
       'Màn hình:', 'Độ phân giải:', 'Tần số quét:', 'Độ phủ màu:',
       'Công nghệ màn hình:', 'Card màn hình:', 'Công nghệ âm thanh:',
       'Cổng giao tiếp:', 'Kết nối không dây:', 'Webcam:', 'Tính năng khác:',
       'Đèn bàn phím:', 'Kích thước:', 'Chất liệu:', 'Thông tin Pin:',
       'Hệ điều hành:', 'Thời điểm ra mắt:', 'Chi tiết', 'Màu', 'Url ảnh',
       'NPU:', 'Hiệu năng xử lý AI (TOPS):', 'Khe đọc thẻ nhớ:',
       'Màn hình cảm ứng:', 'Tản nhiệt:'],
      dtype='object')

In [ ]:
def normalize_column(col):
    """
    Chuẩn hóa tên cột:
    - Bỏ xuống dòng
    - Bỏ dấu ":"
    - Loại bỏ dấu tiếng Việt
    - Chuyển về chữ thường
    - Thay khoảng trắng/ký tự đặc biệt thành "_"
    - Loại bỏ "_" dư thừa ở đầu/cuối
    """
    col = col.replace("\n", " ")                 
    col = col.rstrip(":")                         
    col = unidecode.unidecode(col)                
    col = col.lower()                             
    col = re.sub(r"\W+", "_", col)                
    col = col.strip("_")                          
    return col

In [942]:
df.columns = [normalize_column(col) for col in df.columns]
df.columns = list(map(normalize_column, df.columns))

In [943]:
df.columns

Index(['filename', 'filename_origin', 'time_scraping', 'link', 'tieu_de',
       'da_ban', 'gia_hien_tai', 'gia_cu', 'giam_gia', 'cong_nghe_cpu',
       'so_nhan', 'so_luong', 'toc_do_cpu', 'toc_do_toi_da', 'ram', 'loai_ram',
       'toc_do_bus_ram', 'ho_tro_ram_toi_da', 'o_cung', 'man_hinh',
       'do_phan_giai', 'tan_so_quet', 'do_phu_mau', 'cong_nghe_man_hinh',
       'card_man_hinh', 'cong_nghe_am_thanh', 'cong_giao_tiep',
       'ket_noi_khong_day', 'webcam', 'tinh_nang_khac', 'den_ban_phim',
       'kich_thuoc', 'chat_lieu', 'thong_tin_pin', 'he_dieu_hanh',
       'thoi_diem_ra_mat', 'chi_tiet', 'mau', 'url_anh', 'npu',
       'hieu_nang_xu_ly_ai_tops', 'khe_doc_the_nho', 'man_hinh_cam_ung',
       'tan_nhiet'],
      dtype='object')

In [944]:
df = df.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)

C:\Users\ACER\AppData\Local\Temp\ipykernel_36648\4019745350.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)


In [945]:
df.head()

,filename,filename_origin,time_scraping,link,tieu_de,da_ban,gia_hien_tai,gia_cu,giam_gia,cong_nghe_cpu,so_nhan,so_luong,toc_do_cpu,toc_do_toi_da,ram,loai_ram,toc_do_bus_ram,ho_tro_ram_toi_da,o_cung,man_hinh,do_phan_giai,tan_so_quet,do_phu_mau,cong_nghe_man_hinh,card_man_hinh,cong_nghe_am_thanh,cong_giao_tiep,ket_noi_khong_day,webcam,tinh_nang_khac,den_ban_phim,kich_thuoc,chat_lieu,thong_tin_pin,he_dieu_hanh,thoi_diem_ra_mat,chi_tiet,mau,url_anh,npu,hieu_nang_xu_ly_ai_tops,khe_doc_the_nho,man_hinh_cam_ung,tan_nhiet
0,0220042002813__l265_20250402-105034,l265_20250402-105034,2025-05-30 07:19:18.778722,https://www.thegioididong.com/laptop/asus-fa40...,Laptop Asus TUF Gaming A14 FA401WV R9 AI HX 37...,Đã bán 4,39.990.000₫,44.990.000₫,-11%,AMD Ryzen AI 9 - HX 370,12,24,2GHz,Turbo Boost 5.1 GHz,16 GB,LPDDR5X (Onboard),7500 MHz,Không hỗ trợ nâng cấp,"1 TB SSD M.2 PCIe Gen 4 (Có thể tháo ra, lắp t...","14""",QHD+ (2560 x 1600) 16:10,165 Hz,100% sRGB,G-SyncTấm nền IPSChống chói Anti Glare400 nits,"Card rời - NVIDIA GeForce RTX 4060, 8 GB",Hi-Res AudioDolby Atmos,Jack tai nghe 3.5 mm2 x USB 3.2HDMI1 x USB Typ...,Wi-Fi 6E (802.11ax)Bluetooth 5.3,Full HD WebcamCamera IR,MUX SwitchNVIDIA OptimusBản lề mở 180 độMở khó...,Đơn sắc - Màu trắng,Dài 311 mm - Rộng 227 mm - Dày 19.9 mm - 1.46 kg,Vỏ nhựa - nắp lưng bằng kim loại,"4-cell Li-ion, 73 Wh",Windows 11 Home SL + Office Home & Student vĩn...,2024,Laptop Asus TUF Gaming A14 FA401WV R9 AI HX 37...,Xám,https://cdn.tgdd.vn/Products/Images/44/328948/...,AMD XDNA,Lên đến 50 TOPS,Micro SD,NaN,NaN
1,0220042002563__l66_20250402-105034,l66_20250402-105034,2025-05-30 07:19:14.133143,https://www.thegioididong.com/laptop/lenovo-id...,Laptop Lenovo Ideapad Slim 3 15IRH8 i7 13620H/...,"Đã bán 2,1k",18.190.000₫ *,19.490.000₫,-6%,Intel Core i7 Raptor Lake - 13620H,10,16,2.4GHz,Turbo Boost 4.9 GHz,16 GB,LPDDR5 (Onboard),4800 MHz,Không hỗ trợ nâng cấp,1 TB SSD NVMe PCIe Gen 4,"15.6""",Full HD (1920 x 1080),60Hz,NaN,Low Blue LightTấm nền IPSChống chói Anti Glare...,Card tích hợp - Intel UHD Graphics,Dolby Audio,Jack tai nghe 3.5 mm2 x USB 3.2HDMI1x USB-C 3....,Bluetooth 5.2Wi-Fi 6 (802.11ax),HD webcam,TPM 2.0Độ bền chuẩn quân đội MIL STD 810HBản l...,Không có đèn,Dài 359.3 mm - Rộng 235 mm - Dày 17.9 mm - 1.6...,Vỏ nhựa,47 Wh,Windows 11 Home SL,2023,Không thể bỏ lỡ mẫu laptop Lenovo Ideapad Slim...,Xám,https://cdn.tgdd.vn/Products/Images/44/322073/...,NaN,NaN,SD,NaN,NaN
2,0220042002272__l46_20250402-105034,l46_20250402-105034,2025-05-30 07:19:17.206955,https://www.thegioididong.com/laptop/acer-aspi...,Laptop Acer Aspire 5 A515 58P 34RJ i3 1315U/8G...,"Đã bán 1,8k",NaN,NaN,NaN,Intel Core i3 Raptor Lake - 1315U,6,8,1.2GHz,Turbo Boost 4.5 GHz,8 GB,LPDDR5 (Onboard),4800 MHz,Không hỗ trợ nâng cấp,"512 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...","15.6""",Full HD (1920 x 1080),60Hz,NaN,Tấm nền IPSAcer ComfyView250 nits,Card tích hợp - Intel UHD Graphics,Stereo speakers,Jack tai nghe 3.5 mm2 x USB 3.2HDMI,Wi-Fi 6E (802.11ax)Bluetooth 5.1,HD webcam,NaN,Không có đèn,Dài 362.9 mm - Rộng 237.5 mm - Dày 17.99 mm - ...,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2023,Laptop Acer Aspire 5 A515 58P 34RJ i3 1315U (N...,Xám,https://cdn.tgdd.vn/Products/Images/44/307185/...,NaN,NaN,NaN,NaN,1 x Quạt
3,0220042002367__l9_20250402-105034,l9_20250402-105034,2025-05-30 07:19:15.889382,https://www.thegioididong.com/laptop/asus-vivo...,Laptop Asus Vivobook 15 X1504ZA i3 1215U/8GB/5...,"Đã bán 10,3k",10.690.000₫ *,12.290.000₫,-13%,Intel Core i3 Alder Lake - 1215U,6,8,1.2GHz,Turbo Boost 4.4 GHz,8 GB,DDR4 2 khe (1 khe 8 GB onboard + 1 khe trống),3200 MHz,16 GB,512 GB SSD NVMe PCIe,"15.6""",Full HD (1920 x 1080),60Hz,45% NTSC,Chống chói Anti GlareLED Backlit,Card tích hợp - Intel UHD Graphics,SonicMaster audio,1 x USB 2.0Jack tai nghe 3.5 mm2 x USB 3.2HDMI...,Wi-Fi 6E (802.11ax)Bluetooth 5.3,HD webcam,Độ bền chuẩn quân đội MIL STD 810HBản lề mở 18...,Không có đèn,Dài 359.7 mm - Rộng 232.5 mm - Dày 17.9 mm - 1...,Vỏ nhựa,"3-cell, 42Wh",Wind

**cong_nghe_cpu**

In [ ]:
def extract_columns(cpu_string):
    """
    Trích xuất thông tin từ chuỗi mô tả CPU:
    - Hãng sản xuất (cpu_manufacturer)
    - Dòng sản phẩm (cpu_series)
    - Kiến trúc (cpu_architecture)
    - Model cụ thể (cpu_model)
    """
    result = {
        "cpu_manufacturer": np.nan,
        "cpu_series": np.nan,
        "cpu_architecture": np.nan,
        "cpu_model": np.nan,
    }

    if isinstance(cpu_string, float) and np.isnan(cpu_string):
        return result

    cpu_string = cpu_string.strip()

    # Tìm nhà sản xuất
    manufacturer_pattern = r"(Intel|AMD|Apple|Qualcomm|Celeron|Snapdragon)"
    manufacturer_match = re.search(manufacturer_pattern, cpu_string, re.IGNORECASE)
    if manufacturer_match:
        manufacturer = manufacturer_match.group(1).lower()
        if manufacturer == "celeron":
            result["cpu_manufacturer"] = "intel"
        elif manufacturer == "snapdragon":
            result["cpu_manufacturer"] = "qualcomm"
        else:
            result["cpu_manufacturer"] = manufacturer

    # Tìm series CPU
    series_pattern = (
        r"(Core\s+Ultra\s+\w+|Core\s+\w+|Ryzen\s+AI\s+\w+|Ryzen\s+\w+|"
        r"X\s+Elite|X\s+Plus|X|M\d+\s*(?:Pro|Max)?|Celeron|Snapdragon\s+\w+)"
    )
    series_match = re.search(series_pattern, cpu_string, re.IGNORECASE)
    if series_match:
        result["cpu_series"] = series_match.group(1).strip().lower()

    # Tìm kiến trúc CPU (architecture)
    architecture_pattern = (
        r"(Raptor\s+Lake|Alder\s+Lake|Meteor\s+Lake|Lunar\s+Lake|"
        r"Arrow\s+Lake|Tiger\s+Lake|chuỗi\s+N)"
    )
    architecture_match = re.search(architecture_pattern, cpu_string, re.IGNORECASE)
    if architecture_match:
        result["cpu_architecture"] = architecture_match.group(1).strip().lower()

    # Tìm model cụ thể
    model_pattern = r"-?\s*([^-]+)$"
    model_match = re.search(model_pattern, cpu_string)
    if model_match:
        model = model_match.group(1).strip()
        if result["cpu_manufacturer"] == "qualcomm" and "X1" in cpu_string:
            model_start = cpu_string.index("X1")
            model = cpu_string[model_start:].strip()
        result["cpu_model"] = model

    return result

In [947]:
extracted = df['cong_nghe_cpu'].apply(extract_columns)
df_extracted = pd.json_normalize(extracted)
df = pd.concat([df, df_extracted], axis=1)

In [948]:
df.head()

,filename,filename_origin,time_scraping,link,tieu_de,da_ban,gia_hien_tai,gia_cu,giam_gia,cong_nghe_cpu,so_nhan,so_luong,toc_do_cpu,toc_do_toi_da,ram,loai_ram,toc_do_bus_ram,ho_tro_ram_toi_da,o_cung,man_hinh,do_phan_giai,tan_so_quet,do_phu_mau,cong_nghe_man_hinh,card_man_hinh,cong_nghe_am_thanh,cong_giao_tiep,ket_noi_khong_day,webcam,tinh_nang_khac,den_ban_phim,kich_thuoc,chat_lieu,thong_tin_pin,he_dieu_hanh,thoi_diem_ra_mat,chi_tiet,mau,url_anh,npu,hieu_nang_xu_ly_ai_tops,khe_doc_the_nho,man_hinh_cam_ung,tan_nhiet,cpu_manufacturer,cpu_series,cpu_architecture,cpu_model
0,0220042002813__l265_20250402-105034,l265_20250402-105034,2025-05-30 07:19:18.778722,https://www.thegioididong.com/laptop/asus-fa40...,Laptop Asus TUF Gaming A14 FA401WV R9 AI HX 37...,Đã bán 4,39.990.000₫,44.990.000₫,-11%,AMD Ryzen AI 9 - HX 370,12,24,2GHz,Turbo Boost 5.1 GHz,16 GB,LPDDR5X (Onboard),7500 MHz,Không hỗ trợ nâng cấp,"1 TB SSD M.2 PCIe Gen 4 (Có thể tháo ra, lắp t...","14""",QHD+ (2560 x 1600) 16:10,165 Hz,100% sRGB,G-SyncTấm nền IPSChống chói Anti Glare400 nits,"Card rời - NVIDIA GeForce RTX 4060, 8 GB",Hi-Res AudioDolby Atmos,Jack tai nghe 3.5 mm2 x USB 3.2HDMI1 x USB Typ...,Wi-Fi 6E (802.11ax)Bluetooth 5.3,Full HD WebcamCamera IR,MUX SwitchNVIDIA OptimusBản lề mở 180 độMở khó...,Đơn sắc - Màu trắng,Dài 311 mm - Rộng 227 mm - Dày 19.9 mm - 1.46 kg,Vỏ nhựa - nắp lưng bằng kim loại,"4-cell Li-ion, 73 Wh",Windows 11 Home SL + Office Home & Student vĩn...,2024,Laptop Asus TUF Gaming A14 FA401WV R9 AI HX 37...,Xám,https://cdn.tgdd.vn/Products/Images/44/328948/...,AMD XDNA,Lên đến 50 TOPS,Micro SD,NaN,NaN,amd,ryzen ai 9,NaN,HX 370
1,0220042002563__l66_20250402-105034,l66_20250402-105034,2025-05-30 07:19:14.133143,https://www.thegioididong.com/laptop/lenovo-id...,Laptop Lenovo Ideapad Slim 3 15IRH8 i7 13620H/...,"Đã bán 2,1k",18.190.000₫ *,19.490.000₫,-6%,Intel Core i7 Raptor Lake - 13620H,10,16,2.4GHz,Turbo Boost 4.9 GHz,16 GB,LPDDR5 (Onboard),4800 MHz,Không hỗ trợ nâng cấp,1 TB SSD NVMe PCIe Gen 4,"15.6""",Full HD (1920 x 1080),60Hz,NaN,Low Blue LightTấm nền IPSChống chói Anti Glare...,Card tích hợp - Intel UHD Graphics,Dolby Audio,Jack tai nghe 3.5 mm2 x USB 3.2HDMI1x USB-C 3....,Bluetooth 5.2Wi-Fi 6 (802.11ax),HD webcam,TPM 2.0Độ bền chuẩn quân đội MIL STD 810HBản l...,Không có đèn,Dài 359.3 mm - Rộng 235 mm - Dày 17.9 mm - 1.6...,Vỏ nhựa,47 Wh,Windows 11 Home SL,2023,Không thể bỏ lỡ mẫu laptop Lenovo Ideapad Slim...,Xám,https://cdn.tgdd.vn/Products/Images/44/322073/...,NaN,NaN,SD,NaN,NaN,intel,core i7,raptor lake,13620H
2,0220042002272__l46_20250402-105034,l46_20250402-105034,2025-05-30 07:19:17.206955,https://www.thegioididong.com/laptop/acer-aspi...,Laptop Acer Aspire 5 A515 58P 34RJ i3 1315U/8G...,"Đã bán 1,8k",NaN,NaN,NaN,Intel Core i3 Raptor Lake - 1315U,6,8,1.2GHz,Turbo Boost 4.5 GHz,8 GB,LPDDR5 (Onboard),4800 MHz,Không hỗ trợ nâng cấp,"512 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...","15.6""",Full HD (1920 x 1080),60Hz,NaN,Tấm nền IPSAcer ComfyView250 nits,Card tích hợp - Intel UHD Graphics,Stereo speakers,Jack tai nghe 3.5 mm2 x USB 3.2HDMI,Wi-Fi 6E (802.11ax)Bluetooth 5.1,HD webcam,NaN,Không có đèn,Dài 362.9 mm - Rộng 237.5 mm - Dày 17.99 mm - ...,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2023,Laptop Acer Aspire 5 A515 58P 34RJ i3 1315U (N...,Xám,https://cdn.tgdd.vn/Products/Images/44/307185/...,NaN,NaN,NaN,NaN,1 x Quạt,intel,core i3,raptor lake,1315U
3,0220042002367__l9_20250402-105034,l9_20250402-105034,2025-05-30 07:19:15.889382,https://www.thegioididong.com/laptop/asus-vivo...,Laptop Asus Vivobook 15 X1504ZA i3 1215U/8GB/5...,"Đã bán 10,3k",10.690.000₫ *,12.290.000₫,-13%,Intel Core i3 Alder Lake - 1215U,6,8,1.2GHz,Turbo Boost 4.4 GHz,8 GB,DDR4 2 khe (1 khe 8 GB onboard + 1 khe trống),3200 MHz,16 GB,512 GB SSD NVMe PCIe,"15.6""",Full HD (1920 x 1080),60Hz,45% NTSC,Chống chói Anti GlareLED Backlit,Card tích hợp - Intel UHD Graphics,SonicMaster audio,1 x USB 2.0Jack tai nghe 3.5 mm2 x USB 3.2HDMI...,Wi-Fi 6E (802.11ax)Bluetooth 5.3,HD w

**toc_do_cpu**

In [949]:
def standardize_toc_do_cpu(df):
    """
    Chuẩn hóa cột toc_do_cpu thành số thực (GHz),
    Chuyển GB/s của Apple M-series sang GHz dựa trên loại CPU.

    Parameters:
    df (pandas.DataFrame): DataFrame chứa cột toc_do_cpu và cong_nghe_cpu

    Returns:
    pandas.Series: Cột toc_do_cpu đã được chuẩn hóa về GHz (float), giá trị không xác định là np.nan
    """
    def clean_cpu_speed(row):
        value = row['toc_do_cpu']
        cpu_type = str(row['cong_nghe_cpu']).lower()  # Đảm bảo là chuỗi
        
        if pd.isna(value) or value == '':
            return np.nan
        
        value = str(value).strip()
        
        # Apple M-series GB/s → GHz
        if 'gb/s' in value.lower():
            if 'apple m1' in cpu_type:
                return 3.0
            elif 'apple m2' in cpu_type:
                return 3.2
            elif 'apple m3' in cpu_type:
                return 3.4
            elif 'apple m4 pro' in cpu_type:
                return 3.8
            elif 'apple m4 max' in cpu_type:
                return 4.0
            elif 'apple m4' in cpu_type:
                return 3.5
            return np.nan
        
        # Xử lý GHz thông thường
        try:
            value = value.replace('GHz', '').replace('ghz', '').strip()
            speed = float(value)
            if 0.5 <= speed <= 5.0:
                return speed
            return np.nan
        except (ValueError, TypeError):
            return np.nan

    return df.apply(clean_cpu_speed, axis=1)


In [950]:
df['toc_do_cpu'] = standardize_toc_do_cpu(df)


**toc_do_toi_da**

In [952]:
def standardize_toc_do_toi_da(df):
    """
    Chuẩn hóa cột toc_do_toi_da thành số thực (GHz).
    - Xử lý 'Turbo Boost', đơn vị GHz
    - Ánh xạ Apple M-series nếu toc_do_toi_da là NaN
    - Các giá trị không hợp lệ sẽ được gán np.nan
    """

    def clean_max_speed(row):
        value = row['toc_do_toi_da']
        cpu_type = str(row.get('cong_nghe_cpu', '')).lower()

        # Nếu giá trị là NaN → thử ánh xạ theo Apple M-series
        if pd.isna(value) or str(value).strip().lower() == 'hãng không công bố':
            if 'apple m1' in cpu_type:
                return 3.2
            elif 'apple m2' in cpu_type:
                return 3.5
            elif 'apple m3' in cpu_type:
                return 4.05
            elif 'apple m4 pro' in cpu_type:
                return 4.5
            elif 'apple m4 max' in cpu_type:
                return 4.5
            elif 'apple m4' in cpu_type:
                return 4.4
            return np.nan

        value_str = str(value).lower().replace('ghz', '').replace('turbo boost', '').strip()
        try:
            speed = float(value_str)
            if 2.0 <= speed <= 6.0:
                return speed
        except (ValueError, TypeError):
            pass

        return np.nan

    return df.apply(clean_max_speed, axis=1)


In [953]:
df['toc_do_toi_da'] = standardize_toc_do_toi_da(df)

**npu**

In [955]:
def standardize_npu_binary(df):
    """
    Chuẩn hóa cột npu thành dạng binary (0/1)
    Xử lý NaN dựa trên cong_nghe_cpu để ánh xạ Apple M-series.
    """

    # Kiểm tra cột cần thiết
    if 'cong_nghe_cpu' not in df.columns:
        raise KeyError("DataFrame phải chứa cột 'cong_nghe_cpu' để ánh xạ Apple M-series.")
    if 'npu' not in df.columns:
        raise KeyError("DataFrame phải chứa cột 'npu'.")

    # Tạo cột tạm để xử lý
    df['npu_processed'] = df['npu']

    # Chuẩn hóa chuỗi: xử lý NaN hoặc "Hãng không công bố"
    is_unknown_or_nan = df['npu'].isna() | (df['npu'].str.strip().str.lower() == 'hãng không công bố')

    df.loc[is_unknown_or_nan & df['cong_nghe_cpu'].str.contains('Apple M', case=False, na=False),
           'npu_processed'] = 'Apple Neural Engine'

    df.loc[is_unknown_or_nan & ~df['cong_nghe_cpu'].str.contains('Apple M', case=False, na=False),
           'npu_processed'] = 'Không có NPU'

    # Chuyển về dạng binary
    df['npu'] = df['npu_processed'].apply(
        lambda x: 1 if str(x).strip() in ['Apple Neural Engine', 'Intel AI Boost', 'AMD XDNA', 'Qualcomm Hexagon'] else 0
    )

    # Xóa cột tạm
    df = df.drop(columns=['npu_processed'])

    return df

In [956]:
df = standardize_npu_binary(df)

**hieu_nang_xu_ly_ai_tops**

In [958]:
df['hieu_nang_xu_ly_ai_tops'].unique()

array(['Lên đến 50 TOPS', nan, 'Max 10 TOPS', 'Lên đến 16 TOPS',
       'Lên đến 11 TOPS', 'Lên đến 47 TOPS', 'Lên đến 45 TOPS',
       'Lên đến 48 TOPS', 'Lên đến 40 TOPS', 'Max 13 TOPS'], dtype=object)

In [959]:
def standardize_hieu_nang_xu_ly_ai_tops(df):
    """
    Chuẩn hóa cột hieu_nang_xu_ly_ai_tops thành số thực (TOPS), xóa cột cũ và giữ nguyên tên.
    Nếu npu = 0 thì TOPS = 0.0. Nếu giá trị NaN hoặc 'hãng không công bố', ánh xạ Apple M-series.
    """

    required_columns = ['hieu_nang_xu_ly_ai_tops', 'cong_nghe_cpu', 'npu']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"DataFrame phải chứa cột '{col}'.")

    # Tính giá trị trung bình của các TOPS hợp lệ
    valid_tops = []
    for value in df['hieu_nang_xu_ly_ai_tops']:
        if pd.isna(value):
            continue
        value = str(value).lower().replace('lên đến', '').replace('max', '').replace('tops', '').strip()
        try:
            tops = float(value)
            if 0.0 <= tops <= 50.0:
                valid_tops.append(tops)
        except:
            continue
    mean_tops = np.mean(valid_tops) if valid_tops else 30.0

    def clean_tops(row):
        value = row['hieu_nang_xu_ly_ai_tops']
        cpu_type = str(row['cong_nghe_cpu']).lower()
        npu_status = row['npu']

        # Nếu không có NPU, trả về 0
        if npu_status == 0:
            return 0.0

        # Nếu là NaN hoặc hãng không công bố, dùng ánh xạ theo Apple M-series
        if pd.isna(value) or str(value).strip().lower() == 'hãng không công bố':
            if 'apple m1' in cpu_type:
                return 11.0
            elif 'apple m2' in cpu_type:
                return 15.8
            elif 'apple m3' in cpu_type:
                return 18.0
            elif 'apple m4 pro' in cpu_type:
                return 38.0
            elif 'apple m4 max' in cpu_type:
                return 38.0
            elif 'apple m4' in cpu_type:
                return 38.0
            return 0.0

        # Còn lại, cố gắng parse số
        try:
            value = str(value).lower().replace('lên đến', '').replace('max', '').replace('tops', '').strip()
            tops = float(value)
            if 0.0 <= tops <= 50.0:
                return tops
            return mean_tops
        except:
            return mean_tops

    # Chuẩn hóa
    df['hieu_nang_xu_ly_ai_tops_temp'] = df.apply(clean_tops, axis=1)
    df = df.drop(columns=['hieu_nang_xu_ly_ai_tops'])
    df = df.rename(columns={'hieu_nang_xu_ly_ai_tops_temp': 'hieu_nang_xu_ly_ai_tops'})
    
    return df

In [960]:
df = pd.DataFrame(df)
df = standardize_hieu_nang_xu_ly_ai_tops(df)

**loai_ram**

In [962]:
df['ram'] = df['ram'].astype(str).str.replace(' GB', '')

In [ ]:
def split_loai_ram(ram_string):
    ram_string = str(ram_string).lower().strip()
    
    result = {
        'ram_type': np.nan,
        'ram_slots': 0
    }
    
    ram_type_pattern = r'(ddr\d+|lpddr\d+|lpddr\d+x)'
    ram_type_match = re.search(ram_type_pattern, ram_string, re.IGNORECASE)
    if ram_type_match:
        result['ram_type'] = ram_type_match.group(1).upper()
    

    if 'onboard' in ram_string and 'khe' not in ram_string:
        result['ram_slots'] = 0
    elif '2 khe' in ram_string:
        result['ram_slots'] = 2
    elif '1 khe' in ram_string or ('onboard' in ram_string and 'khe' in ram_string):
        result['ram_slots'] = 1
    else:
        result['ram_slots'] = np.nan
    
    return result

In [964]:
ram_info = df['loai_ram'].apply(split_loai_ram).apply(pd.Series)
df[['ram_type', 'ram_slots']] = ram_info[['ram_type', 'ram_slots']]

**ram_slots**

In [965]:
df.loc[df['cpu_manufacturer'].str.lower() == 'apple', 'ram_slots'] = 0.0

**ho_tro_ram_toi_da**

In [967]:
df['ho_tro_ram_toi_da'] = df['ho_tro_ram_toi_da'].astype(str).str.replace(' GB', '')
df.rename(columns={'ho_tro_ram_toi_da': 'max_ram_support'}, inplace=True)

In [969]:
df.rename(columns={'ram': 'ram_storage'}, inplace=True)


**max_ram_support**

In [970]:
condition = df['max_ram_support'] == 'Không hỗ trợ nâng cấp'
df.loc[condition, 'max_ram_support'] = df.loc[condition, 'ram_storage']

**o_cung**

In [ ]:
def parse_storage_size(size_str):
    """
    Chuyển đổi chuỗi dung lượng thành số GB.
    Ví dụ: '512GB' → 512, '1TB' → 1024
    """
    size_str = size_str.lower()
    match = re.search(r'(\d+(?:\.\d+)?)\s*(gb|tb)', size_str)
    if match:
        size = float(match.group(1))
        unit = match.group(2)
        if unit == "tb":
            size *= 1024
        return size
    return np.nan


def split_o_cung_full(text):
    """
    Phân tích thông tin ổ cứng từ mô tả:
    - drive_type: loại ổ (HDD/SSD)
    - interface: giao tiếp (NVMe/SATA/PCIe)
    - storage_gb: tổng dung lượng ổ
    - storage_max_support: dung lượng tối đa hỗ trợ (nếu có)
    """
    if not isinstance(text, str) or not text.strip():
        return {
            "drive_type": np.nan,
            "interface": np.nan,
            "storage_gb": np.nan,
            "storage_max_support": np.nan,
        }

    # Trích xuất phần trong ngoặc nếu có
    bracket_match = re.search(r"\((.*?)\)", text)
    bracket_content = bracket_match.group(1) if bracket_match else None

    # Loại bỏ phần ngoặc để xử lý phần chính
    main_text = re.sub(r"\(.*?\)", "", text).lower()
    parts = re.split(r"\+|,|và", main_text)

    drive_types = []
    total_storage = 0
    all_interfaces = []

    for part in parts:
        part = part.strip()

        # Xác định loại ổ
        if "ssd" in part:
            drive_types.append("SSD")
        if "hdd" in part:
            drive_types.append("HDD")

        # Giao tiếp
        if "nvme" in part:
            all_interfaces.append("NVMe")
        if "sata" in part:
            all_interfaces.append("SATA")
        if "pcie" in part:
            all_interfaces.append("PCIe")

        # Kích thước
        size = parse_storage_size(part)
        if not np.isnan(size):
            total_storage += size

    # Giao tiếp duy nhất, giữ thứ tự xuất hiện
    unique_interfaces = list(dict.fromkeys(all_interfaces))
    final_interface = ", ".join(unique_interfaces) if unique_interfaces else np.nan

    # Dung lượng tối đa hỗ trợ
    max_upgrade = np.nan
    if bracket_content:
        matches = re.findall(r"(\d+(?:\.\d+)?)\s*(gb|tb)", bracket_content.lower())
        sizes = []
        for num, unit in matches:
            size = float(num)
            if unit == "tb":
                size *= 1024
            sizes.append(size)
        if sizes:
            max_upgrade = max(sizes)
        elif total_storage > 0:
            max_upgrade = total_storage

    return {
        "drive_type": ", ".join(sorted(set(drive_types))) if drive_types else np.nan,
        "interface": final_interface,
        "storage_gb": total_storage if total_storage > 0 else np.nan,
        "storage_max_support": max_upgrade,
    }


In [ ]:
storage_info = df['o_cung'].apply(split_o_cung_full).apply(pd.Series)
df = df.join(storage_info)

**storage_max_support**

In [975]:
condition = df['storage_max_support'].isna()
df.loc[condition, 'storage_max_support'] = df.loc[condition, 'storage_gb']

**man_hinh**

In [ ]:
def standardize_display_size(value):
    """
    Chuẩn hóa kích thước màn hình:
    - Chuyển từ chuỗi dạng '15.6"' hoặc '2x14' về dạng số (float).
    - Nếu là dạng nhân (2x14) thì trả về tích (28.0).
    """
    if pd.isna(value):
        return np.nan

    s = str(value).replace('"', "").strip()

    # Kiểm tra dạng "2x14" hoặc "3 x 15.6"
    match = re.match(r"(\d+(?:\.\d+)?)\s*[x×]\s*(\d+(?:\.\d+)?)", s, flags=re.IGNORECASE)
    if match:
        num1 = float(match.group(1))
        num2 = float(match.group(2))
        return num1 * num2

    # Nếu không phải dạng nhân, thử ép kiểu float
    try:
        return float(s)
    except ValueError:
        return np.nan


# Chuẩn hóa và đổi tên cột
df["display_size"] = df["man_hinh"].apply(standardize_display_size)
df.drop(columns=["man_hinh"], inplace=True)

**resolution**

In [ ]:
def standardize_resolution_to_pixels(resolution):
    """
    Chuẩn hóa chuỗi độ phân giải về kích thước pixel: (width, height).
    Hỗ trợ các dạng phổ biến như '1920 x 1080', 'Full HD', 'QHD+', 'Retina', v.v.
    """
    if not resolution or str(resolution).lower().strip() in ["nan", ""]:
        return 0, 0  # Giá trị mặc định cho trường hợp rỗng

    resolution = str(resolution).lower().strip()

    # Mapping các tên độ phân giải phổ biến sang kích thước
    resolution_map = {
        "hd": (1366, 768),
        "full hd": (1920, 1080),
        "full hd+": (1920, 1200),
        "wuxga": (1920, 1200),
        "qhd": (2560, 1440),
        "qhd+": (2560, 1600),
        "wqxga": (2560, 1600),
        "wqxga+": (2560, 1600),
        "2.2k": (2560, 1600),
        "2.5k": (2560, 1600),
        "2.8k": (2880, 1800),
        "3.2k": (3200, 2000),
        "4k/uhd+": (3840, 2400),
        "retina": (2560, 1600),
        "liquid retina": (2560, 1664)
    }

    # Kiểm tra chuỗi định dạng pixel trực tiếp (vd: "1920 x 1080")
    pixel_pattern = r"(\d+)\s*[x×]\s*(\d+)"
    match = re.search(pixel_pattern, resolution)
    if match:
        width, height = int(match.group(1)), int(match.group(2))
        return width, height

    # Kiểm tra khớp với các tên độ phân giải phổ biến
    for key, (width, height) in resolution_map.items():
        if key in resolution:
            return width, height

    return 0, 0  # Không xác định được

df[["display_width", "display_height"]] = (
    df["do_phan_giai"]
    .apply(standardize_resolution_to_pixels)
    .apply(pd.Series)
    .astype(float)
)

**tan_so_quet**

In [ ]:
def standardize_refresh_rate(row):
    """
    Chuẩn hóa tần số quét (refresh rate) về dạng số float (Hz).
    - Nếu là khoảng (e.g., '48 - 120Hz') → lấy số lớn hơn.
    - Nếu là 'hãng không công bố' và có Liquid Retina XDR → 120 Hz, ngược lại → 60 Hz.
    - Nếu không khớp → mặc định 60.0.
    """
    refresh_rate = str(row.get("tan_so_quet", "")).lower().strip()
    resolution = str(row.get("do_phan_giai", "")).lower().strip()

    # Trường hợp "hãng không công bố" (MacBook)
    if refresh_rate == "hãng không công bố":
        if "liquid retina xdr" in resolution:
            return 120.0  # ProMotion MacBook Pro
        return 60.0

    # Regex: bắt cả dạng '60Hz', 'up to 120 Hz', '48 - 120 Hz', v.v.
    pattern = r"(\d+\.?\d*)\s*(?:hz|-|to|up\s*to)?\s*(\d+\.?\d*)?\s*hz?"
    match = re.search(pattern, refresh_rate, flags=re.IGNORECASE)

    if match:
        val1 = float(match.group(1))
        val2 = float(match.group(2)) if match.group(2) else None
        return max(val1, val2) if val2 else val1

    return 60.0  # Mặc định nếu không xác định được

# Áp dụng hàm cho cột tan_so_quet
df['tan_so_quet'] = df.apply(standardize_refresh_rate, axis=1)
df.rename(columns={'tan_so_quet': 'refresh_rate'}, inplace=True)

**do_phu_mau**

In [ ]:
def standardize_color_gamut_score(row):
    """
    Chuẩn hóa độ phủ màu thành một điểm số chuẩn hóa (theo thang 0.0–1.0).
    Các nguồn được quy đổi về chuẩn NTSC để dễ so sánh.
    """
    gamut = str(row.get("do_phu_mau", "")).lower().strip()
    resolution = str(row.get("do_phan_giai", "")).lower().strip()
    tech = str(row.get("cong_nghe_man_hinh", "")).lower().strip()

    # Trường hợp hãng không công bố – Ước lượng theo MacBook, OLED, IPS, v.v.
    if gamut == "hãng không công bố":
        # MacBook
        if "liquid retina xdr" in resolution or "promotion" in tech or "wide color (p3)" in tech:
            return 0.80  # ~100% DCI-P3
        if "liquid retina" in resolution or "retina" in resolution:
            return 0.72  # ~100% sRGB

        # Màn hình cao cấp
        if any(keyword in tech for keyword in ["oled", "mini led", "displayhdr true black", "dolby vision"]):
            return 0.80
        if "oled" in resolution:
            return 0.80

        # Ước lượng theo độ sáng
        nits_match = re.search(r"(\d+)\s*nits", tech)
        nits = float(nits_match.group(1)) if nits_match else 0

        if "ips" in tech and nits >= 300:
            return 0.72  # sRGB
        if "wva" in tech or "tn" in tech or nits <= 250:
            return 0.45  # thường là 45% NTSC

        return 0.45  # Mặc định bảo thủ

    # Trích xuất dữ liệu phần trăm + không gian màu
    pattern = r"(\d+\.?\d*)\s*%\s*(dci-p3|srgb|ntsc|adobe rgb)"
    matches = re.findall(pattern, gamut, flags=re.IGNORECASE)

    if not matches:
        return 0.45  # Mặc định nếu không nhận diện được

    # Quy đổi sang chuẩn NTSC
    score_map = {
        "ntsc": lambda x: x / 100,
        "srgb": lambda x: (x / 100) * 0.72,
        "dci-p3": lambda x: (x / 100) * 0.80,
        "adobe rgb": lambda x: (x / 100) * 0.78
    }

    scores = [
        score_map.get(gamut_type.lower(), lambda x: 0.45)(float(percent))
        for percent, gamut_type in matches
    ]

    return max(scores) if scores else 0.45

df['color_gamut_score'] = np.round(df.apply(standardize_color_gamut_score, axis=1), 2)


**card_man_hinh**

In [ ]:
def extract_vga_info(row):
    """
    Trích xuất thông tin VGA từ chuỗi 'card_man_hinh'.
    Trả về loại card (rời/tích hợp), hãng sản xuất và VRAM.
    """
    vga_str = str(row.get("card_man_hinh", "")).lower().strip()

    # Kết quả mặc định
    result = {
        "vga_type": "Card tích hợp",  # Mặc định
        "vga_brand": np.nan,
        "vram": np.nan
    }

    # Trích xuất VRAM
    vram_match = re.search(r"(\d+)\s*gb", vga_str)
    if vram_match:
        result["vram"] = float(vram_match.group(1))

    # Phân loại card rời
    if "card rời" in vga_str:
        result["vga_type"] = "Card rời"
        if "nvidia" in vga_str:
            result["vga_brand"] = "NVIDIA"
        elif "amd radeon rx" in vga_str:
            result["vga_brand"] = "AMD"
    else:  # Card tích hợp
        if "intel" in vga_str:
            result["vga_brand"] = "Intel"
        elif "amd radeon" in vga_str:
            result["vga_brand"] = "AMD"
        elif "nhân gpu" in vga_str or "apple" in vga_str:
            result["vga_brand"] = "Apple"
        elif "qualcomm" in vga_str:
            result["vga_brand"] = "Qualcomm"

    return pd.Series(result)

# Áp dụng hàm
vga_df = df.apply(extract_vga_info, axis=1)
df = pd.concat([df, vga_df], axis=1)

**ket_noi_khong_day**

In [ ]:
def extract_wireless_tech(row):
    """
    Trích xuất thông tin Wi-Fi và Bluetooth từ chuỗi 'ket_noi_khong_day'.

    Trả về:
    - wifi_version: '7', '6E', '6', 'Legacy', hoặc NaN
    - bluetooth_version: phiên bản Bluetooth (vd: '5.2'), hoặc NaN
    """
    text = str(row.get("ket_noi_khong_day", "")).lower().strip()

    result = {
        "wifi_version": np.nan,
        "bluetooth_version": np.nan
    }

    # Wi-Fi version
    if any(x in text for x in ["802.11be", "wi-fi 7"]):
        result["wifi_version"] = "7"
    elif "wi-fi 6e" in text:
        result["wifi_version"] = "6E"
    elif any(x in text for x in ["802.11ax", "wi-fi 6"]):
        result["wifi_version"] = "6"
    elif "802.11 a/b/g/n/ac" in text:
        result["wifi_version"] = "Legacy"

    # Bluetooth version
    match_bt = re.search(r"bluetooth\s*([\d\.]+)", text)
    if match_bt:
        result["bluetooth_version"] = match_bt.group(1)
    elif "bluetooth" in text:
        result["bluetooth_version"] = np.nan

    return pd.Series(result)

In [987]:
df[['wifi_version', 'bluetooth_version']] = df.apply(extract_wireless_tech, axis=1)

**toc_do_bus_ram**

In [ ]:
# Từ điển ánh xạ tần số RAM theo dòng CPU Apple
ram_freq_map = {
    'm1': 4266, 'm1 pro': 4266, 'm1 max': 4266, 'm1 ultra': 4266,
    'm2': 6400, 'm2 pro': 6400, 'm2 max': 6400, 'm2 ultra': 6400,
    'm3': 6400, 'm3 pro': 6400, 'm3 max': 6400,
    'm4': 7500, 'm4 pro': 7500, 'm4 max': 7500
}

def standardize_toc_do_bus_ram(row):
    """
    Chuẩn hóa cột tốc độ RAM: nếu giá trị bị thiếu, dùng ánh xạ từ cpu_series.
    Nếu có dạng chuỗi, chuẩn hóa và chuyển thành số nguyên.
    """
    value = row["toc_do_bus_ram"]
    cpu_series = str(row.get("cpu_series", "")).lower()

    # Nếu bị thiếu hoặc đang cập nhật
    if pd.isna(value) or str(value).lower().strip() == "đang cập nhật":
        return ram_freq_map.get(cpu_series, np.nan)

    if isinstance(value, str):
        value = value.lower()
        value = re.sub(r"(từ|lên đến|\(hãng công bố\)|mhz)", "", value)
        value = re.sub(r"[^\d]", "", value).strip()  # Bỏ mọi ký tự không phải số

        try:
            return int(value)
        except ValueError:
            return ram_freq_map.get(cpu_series, np.nan)

    return value

**kich_thuoc**

In [ ]:
def parse_dimensions(row):
    """
    Trích xuất kích thước vật lý từ chuỗi 'kich_thuoc' theo định dạng:
    'Dài ... mm - Rộng ... mm - Dày ... mm - ... kg'

    Trả về: width_mm, height_mm, depth_mm, weight_kg
    """
    text = str(row.get("kich_thuoc", "")).lower()

    # Chuẩn hóa chuỗi
    text = (
        text.replace(" - ", "|")
            .replace("dài ", "")
            .replace("rộng ", "")
            .replace("dày ", "")
            .replace(" mm đến ", "~")
            .replace(" kg", "")
    )

    parts = text.split("|")

    if len(parts) != 4:
        return pd.Series([np.nan] * 4)

    try:
        width = float(parts[0].replace(" mm", "").strip())
        height = float(parts[1].replace(" mm", "").strip())

        # Xử lý độ dày (depth)
        depth_str = parts[2].replace(" mm", "").strip()
        if "~" in depth_str:
            depth = float(depth_str.split("~")[0].strip())
        else:
            depth = float(depth_str)

        weight = float(parts[3].strip())

        return pd.Series([width, height, depth, weight])
    except Exception:
        return pd.Series([np.nan] * 4)

# Áp dụng hàm và tạo các cột mới
df[['width_mm', 'height_mm', 'depth_mm', 'weight_kg']] = df.apply(parse_dimensions, axis=1)

**he_dieu_hanh**

In [ ]:
def extract_os(row):
    """
    Chuẩn hóa hệ điều hành từ cột 'he_dieu_hanh'.
    Trả về 'os_version' với các giá trị: Windows 11, Windows 10, MacOS, Chrome OS, hoặc NaN.
    """
    os_raw = str(row.get("he_dieu_hanh", "")).lower().strip()

    if "windows 11" in os_raw:
        return pd.Series({"os_version": "Windows 11"})
    elif "windows 10" in os_raw:
        return pd.Series({"os_version": "Windows 10"})
    elif "macos" in os_raw:
        return pd.Series({"os_version": "MacOS"})
    elif "chrome os" in os_raw or "chromeos" in os_raw:
        return pd.Series({"os_version": "Chrome OS"})
    else:
        return pd.Series({"os_version": np.nan})

In [994]:
os_df = df.apply(extract_os, axis=1)
df = pd.concat([df, os_df], axis=1)

**thong_tin_pin**

In [ ]:
def extract_battery_capacity(row):
    """
    Trích xuất dung lượng pin (Wh) từ cột 'thong_tin_pin'.
    Trả về Series với 'battery_capacity_wh'.
    """
    battery_info = str(row.get("thong_tin_pin", "")).lower().strip()
    
    match = re.search(r'(\d+\.?\d*)\s*wh', battery_info)
    if match:
        return pd.Series({"battery_capacity_wh": float(match.group(1))})
    
    return pd.Series({"battery_capacity_wh": np.nan})

In [ ]:
# Áp dụng hàm
battery_df = df.apply(extract_battery_capacity, axis=1)
df = pd.concat([df, battery_df], axis=1)


**webcam**

In [ ]:
def extract_webcam_resolution(row):
    """
    Trích xuất độ phân giải của webcam từ cột 'webcam'.
    Trả về 'webcam_resolution' là 'Full HD', 'HD', hoặc None.
    """
    webcam_info = str(row.get('webcam', '')).lower().strip()
    
    full_hd_terms = ['full hd', '1080p', 'qhd', '5 mp', '5.0 mp', '9 mp', 'fhd']
    hd_terms = ['hd', '720p', 'camera ir']
    
    if any(term in webcam_info for term in full_hd_terms):
        resolution = 'Full HD'
    elif 'hp truevision webcam' in webcam_info and not any(term in webcam_info for term in full_hd_terms):
        resolution = 'HD'
    elif any(term in webcam_info for term in hd_terms):
        resolution = 'HD'
    else:
        resolution = None
    
    return pd.Series({'webcam_resolution': resolution})

In [999]:
webcam_df = df.apply(extract_webcam_resolution, axis=1)
df = pd.concat([df, webcam_df], axis=1)

**warranty_information**

In [1001]:

def assign_warranty_info(row):
    # Khởi tạo kết quả với warranty_information là danh sách cố định
    result = {'warranty_information': [
        'Hư gì đổi nấy 12 tháng tại 2957 siêu thị toàn quốc',
        'Bảo hành chính hãng laptop 2 năm tại các trung tâm bảo hành hãng'
    ]}
    
    # Trả về pd.Series chứa danh sách các chuỗi
    return pd.Series(result)

**product_state**

In [ ]:
def assign_product_state(row):
    # Khởi tạo kết quả với product_state là danh sách rỗng
    result = {'product_state': []}
    
    # Kiểm tra giá trị os_version trong row
    if row['os_version'] == 'MacOS':
        result['product_state'] = [
            'Máy mới 100%, đầy đủ phụ kiện từ nhà sản xuất',
            'Sản phẩm có mã SA/A (được Apple Việt Nam phân phối chính thức)'
        ]
    else:
        result['product_state'] = [
            'Nguyên hộp, đầy đủ phụ kiện từ nhà sản xuất'
        ]
    
    # Trả về pd.Series chứa danh sách các chuỗi
    return pd.Series(result)

In [1003]:
product_state_df = df.apply(assign_product_state, axis=1)   

In [1004]:
# Kết hợp các cột vào DataFrame
df = pd.concat([df, product_state_df], axis=1)

**url_anh**

In [ ]:
def check_image_url(url):
    """
    Kiểm tra xem URL có dẫn tới một hình ảnh hợp lệ không.
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/91.0.4472.124 Safari/537.36'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)

        # Kiểm tra HTTP status code
        if response.status_code != 200:
            return f"❌ URL không hoạt động (status code: {response.status_code})"

        # Kiểm tra content-type
        content_type = response.headers.get('Content-Type', '').lower()
        if not content_type.startswith('image/'):
            return f"❌ URL không trả về hình ảnh (Content-Type: {content_type})"

        # Kiểm tra tính hợp lệ của hình ảnh
        try:
            img = Image.open(BytesIO(response.content))
            img.verify()  # Xác minh hình ảnh
            return "✅ Hình ảnh hợp lệ và hoạt động bình thường."
        except Exception as e:
            return f"❌ Hình ảnh không hợp lệ hoặc bị hỏng: {e}"

    except requests.exceptions.RequestException as e:
        return f"❌ Lỗi khi truy cập URL: {e}"

Hình ảnh hoạt động bình thường và hợp lệ.


In [ ]:
def select_first_image(text):
    """
    Lấy URL hình ảnh đầu tiên từ chuỗi chứa nhiều URL phân tách bằng dấu ";"
    """
    if not isinstance(text, str) or pd.isna(text) or not text.strip():
        return None

    urls = text.split(";")
    return urls[0].strip() if urls else np.nan

In [1007]:
df['url_anh'] = df['url_anh'].apply(select_first_image)

**chi_tiet**

In [1008]:
def clean_and_split_sentences(detail_text: str) -> list[str]:
    if not isinstance(detail_text, str):
        return []

    # Tách thành câu dựa trên dấu chấm kết thúc câu
    raw_sentences = re.split(r'(?<=[.])\s+', detail_text)
    
    # Loại bỏ các câu chứa "Xem thêm" và loại bỏ khoảng trắng dư
    cleaned_sentences = [sentence.strip() for sentence in raw_sentences if "Xem thêm" not in sentence]
    
    return cleaned_sentences

In [1009]:
# Áp dụng xử lý cho từng dòng
df["chi_tiet"] = df["chi_tiet"].apply(clean_and_split_sentences)

# Kiểm tra kết quả
print(df["chi_tiet"].iloc[0])

['Laptop Asus TUF Gaming A14 FA401WV R9 AI HX 370 (RG062WS) là chiếc laptop hoàn hảo cho cả game thủ lẫn người dùng chuyên nghiệp.', 'Với thiết kế bền bỉ, hiệu năng mạnh mẽ, màn hình sắc nét, và các công nghệ tối ưu, chiếc máy này không chỉ đáp ứng mọi nhu cầu làm việc và giải trí mà còn mang lại giá trị lâu dài.•\xa0Chip AMD Ryzen 9 AI HX 370 với 12 nhân và 24 luồng mang đến khả năng xử lý đa nhiệm vượt trội, lý tưởng cho các laptop gaming cao cấp.', 'Với khả năng Turbo Boost lên đến 5.1 GHz, chip này đảm bảo hiệu năng mạnh mẽ, giúp xử lý mượt mà các tựa game nặng hay phần mềm đồ họa chuyên nghiệp.', 'Công nghệ AI tích hợp không chỉ tối ưu hóa hiệu suất theo nhu cầu sử dụng mà còn giúp tiết kiệm năng lượng, đảm bảo máy hoạt động ổn định trong thời gian dài.', 'Với Ryzen 9 AI HX 370, người dùng có thể trải nghiệm các tựa game AAA, stream nội dung trực tuyến, đồng thời đây cũng là chiếc laptop thiết kế đồ họa phù hợp với công việc sáng tạo.•\xa0NVIDIA GeForce RTX 4060 với dung lượng VRA

**manufacturer**

In [ ]:
def extract_manufacturer(name):
    """
    Trích xuất tên nhà sản xuất từ tên sản phẩm laptop.
    """
    if not isinstance(name, str):
        return np.nan

    name = name.lower().strip()

    prefix_map = {
        "laptop lenovo": "lenovo",
        "laptop hp": "hp",
        "laptop dell": "dell",
        "laptop gigabyte": "gigabyte",
        "laptop acer": "acer",
        "laptop msi": "msi",
        "laptop asus": "asus",
        "laptop macbook": "apple",
        "laptop apple": "apple",
        "laptop samsung": "samsung",
        "laptop lg": "lg"
    }

    for prefix, brand in prefix_map.items():
        if name.startswith(prefix):
            return brand

    return np.nan

In [1011]:
df['manufacturer'] = df['tieu_de'].apply(extract_manufacturer)

In [1012]:
df['is_installment'] = 1

In [1013]:
df['so_luong'] = df['so_luong'].replace('hãng không công bố', np.nan)

In [ ]:
df['time_scraping'] = pd.to_datetime(df['time_scraping'])
df['time_scraping'] = df['time_scraping'].dt.strftime('%Y-%m-%d %H:%M:%S')

**tinh_nang_khac**

In [ ]:
def extract_features(text, features):
    """
    Trích xuất danh sách tính năng có trong `features` xuất hiện trong `text`.

    Parameters:
    - text (str): Chuỗi chứa mô tả sản phẩm.
    - features (list): Danh sách các tính năng cần tìm.

    Returns:
    - list: Danh sách tính năng tìm thấy hoặc np.nan nếu không có.
    """
    if not isinstance(text, str) or not text.strip():
        return np.nan

    pattern = '|'.join(map(re.escape, features))
    matches = re.findall(pattern, text)

    return matches if matches else np.nan


In [1016]:
df['tinh_nang_khac'] = df['tinh_nang_khac'].apply(lambda x: extract_features(x, known_features))


In [1017]:
df.rename(columns={'filename_origin': 'product_id',
                   'tieu_de' : 'name',
                   'url_anh' : 'image',
                   'man_hinh' : 'display_size',
                   'current_storage' : 'laptop_storage',
                   'chi_tiet' : 'key_selling_points',
                   'chat_lieu' : 'material',
                   'cong_nghe_am_thanh' : 'laptop_cong_nghe_am_thanh',
                   'ram' : 'ram_storage',
                   'toc_do_bus_ram' : 'ram_speed',
                    'ho_tro_ram_toi_da' : 'ram_max_support',
                    'battery_capacity_wh' : 'battery_capacity',
                    'webcam_resolution' : 'laptop_camera',
                    'cpu_manufacturer' : 'cpu_brand',
                    'so_nhan' : 'cpu_cores',
                    'so_luong': 'cpu_threads',
                    'gia_hien_tai' : 'discounted_price',
                    'gia_cu' : 'root_price',
                    'gia_ban' : 'price',
                    'tan_so_quet' : 'refresh_rate',
                    'tinh_nang_khac' : 'laptop_special_feature',
                    'cpu_model_number' : 'cpu_model',
                    'vram' : 'vga_vram',
                    'toc_do_cpu': 'cpu_speed',
                    'toc_do_toi_da': 'cpu_max_speed',
                    'man_hinh_cam_ung' :   'cam_ung',
                    'link' : 'url_path',
                    'cong_nghe_man_hinh' : 'display_type',
                    'cong_giao_tiep' :'ports_slots',
                    'mau' : 'laptop_color',
                    'weight_kg' : 'product_weight'
                    
                    
}, inplace=True)

In [1018]:
columns_to_float = [
    'cpu_cores',
    'cpu_threads',
    'ram_storage',
    'ram_speed',
    'max_ram_support',
]

# Chuyển kiểu dữ liệu sang float, nếu lỗi thì thành np.nan
for col in columns_to_float:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')

In [1019]:
# Lấy các cột có kiểu dữ liệu số
numeric_cols = df.select_dtypes(include='number')

# Kiểm tra cột nào có ít nhất một giá trị = 0
cols_with_zero = (numeric_cols == 0).any()

# Lọc ra tên các cột có giá trị 0
columns_with_zero = cols_with_zero[cols_with_zero].index.tolist()

print("Các cột số có ít nhất một giá trị bằng 0:")
print(columns_with_zero)

Các cột số có ít nhất một giá trị bằng 0:
['npu', 'hieu_nang_xu_ly_ai_tops', 'ram_slots']


**price**

In [ ]:
def clean_price(val):
    """
    Làm sạch và chuyển đổi chuỗi giá tiền sang kiểu số nguyên.

    Parameters:
    - val (str|int|float): Giá tiền dạng chuỗi hoặc số.

    Returns:
    - int|float: Giá tiền dạng số nguyên hoặc np.nan nếu không thể chuyển đổi.
    """
    if isinstance(val, str):
        val = val.replace('₫', '').replace('*', '').replace('.', '').replace(',', '').strip()
        try:
            return int(val)
        except ValueError:
            return np.nan
    try:
        return int(val)
    except (ValueError, TypeError):
        return np.nan

In [1021]:
df['root_price'] = df['root_price'].apply(clean_price)
df['discounted_price'] = df['discounted_price'].apply(clean_price)

**da_ban**

In [1022]:
df['da_ban'] = df['da_ban'].apply(
    lambda x: x.replace('Đã bán', '').strip() if isinstance(x, str) else x
)

In [ ]:
def convert_k_to_number(val):
    """
    Chuyển đổi chuỗi có hậu tố 'k' (nghìn) thành số nguyên.

    Ví dụ:
    - '1.2k' → 1200
    - '900'  → 900

    Parameters:
    - val: Chuỗi hoặc số.

    Returns:
    - int: Giá trị số sau khi chuyển đổi, hoặc None nếu lỗi.
    """
    if isinstance(val, str):
        val = val.strip().lower().replace(',', '.')
        if 'k' in val:
            try:
                return int(float(val.replace('k', '')) * 1000)
            except ValueError:
                return None
    try:
        return int(float(val))
    except (ValueError, TypeError):
        return None

**giam_gia**

In [ ]:
def parse_percentage(val):
    """
    Chuyển đổi giá trị phần trăm (có thể có ký hiệu %, dấu phẩy/thập phân, hoặc trong ngoặc) 
    về dạng số thực thập phân (từ 0 đến 1).
    """
    if isinstance(val, str):
        val = val.strip().replace('%', '').replace(',', '.')
        # Dạng "(8%)" → "-8"
        if val.startswith('(') and val.endswith(')'):
            val = '-' + val[1:-1]
    try:
        num = abs(float(val)) / 100  # đảm bảo luôn là dương
        return round(num, 4)
    except:
        return None

In [1025]:
columns_to_lowercase = [
    'do_phu_mau', 'khe_doc_the_nho', 'den_ban_phim', 'material',
    'cpu_model', 'ram_type', 'drive_type', 'interface',
    'vga_type', 'vga_brand', 'wifi_version', 'os_version', 'webcam', 'do_phan_giai'
]

# Chỉ chọn các cột thực sự tồn tại trong DataFrame
existing_columns = [col for col in columns_to_lowercase if col in df.columns]

# Áp dụng chuyển về chữ thường cho từng cột
for col in existing_columns:
    df[col] = df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [1027]:
df['cam_ung'] = df['cam_ung'].apply(
    lambda x: 1 if x == 'Cảm ứng' else 0
)


In [1028]:
df.drop(columns=['webcam', 'he_dieu_hanh', 'kich_thuoc', 'o_cung', 'cong_nghe_cpu', 'loai_ram',
                 'card_man_hinh', 'ket_noi_khong_day','thong_tin_pin' , 'do_phan_giai'], inplace=True)

In [1029]:
df_label = pd.read_csv('tgdd_extracted_v3.csv')

In [1030]:
df_label = df_label[['product_id', 'nhu_cau_su_dung']]

In [1031]:
df = pd.merge(df, df_label, on='product_id', how='left')

In [1032]:
df.drop(columns= 'product_id', inplace=True)
df.rename(columns={ 'filename' : 'product_id'}, inplace=True)

In [1033]:
df.to_csv('the_gioi_di_dong_v5.csv', index=False)